In [ ]:
####################################################
# This is a scratchpad for analysis of intermediate results in the CGMBrush library.
####################################################

# NOTE: Always clear results before checking in a jupyter notebook. This improves diff's.

from cgmbrush.cgmbrush import *
from cgmbrush.plots.plots import *
import matplotlib.colors as colors
from cgmbrush.cosmology import cosmology as cosmo
from cgmbrush.cosmology import halo as halo
import numpy as np

%matplotlib inline


In [ ]:
###################################################
# Profile Mask Visualizer
###################################################
def view_mask(profile: CGMProfile, mass):

    provider = BolshoiProvider()

    # pick some parameters
    redshift = 0 
    resolution = 16
    cellsize = provider.Lbox / (provider.halofieldresolution * resolution)
    print("cellsize (kpc): %s" % (cellsize * 1000))
    fine_mask_len = 20 * resolution
    fine_mask_len_small = 2 * resolution

    comoving_rvir = halo.comoving_rvir(cosmo, mass, redshift)
    print("co-moving virial radius (kpc): %s" % (comoving_rvir * 1000))

    mask = profile.get_mask(mass, comoving_rvir, redshift, resolution, 1, cellsize, fine_mask_len)
    mask_small = profile.get_mask(mass, comoving_rvir, redshift, resolution, 1, cellsize, fine_mask_len_small)
    fig, ax = plt.subplots(1,2,figsize=(24, 12))
    pos = ax[0].imshow(mask)
    pos = ax[1].imshow(mask_small)
    ax[0].title.set_text('{} Profile, M=${:.1e}$'.format(profile.name, mass))
    ax[1].title.set_text('{} Profile (zoomed), M=${:.1e}$'.format(profile.name, mass))
    fig.colorbar(pos, ax=ax)

# Choose a profile and mass
view_mask(PrecipitationProfile(), 2E10) 
view_mask(SphericalTophatProfile(rvir_factor=3), 2E10) 
view_mask(FireProfile(), 2E10) 

In [ ]:
from cgmbrush import *
def compareRadii(mass, z):
    print("Mass {:.1e}, z={}".format(mass, z))
    a = halo.comoving_rvir(cosmo, mass, z)
    b = halo.r200Mz(cosmo, mass, z)
    print('  rVir:\t{:.4f} Mpc'.format(a))
    print('  r200:\t{:.4f} Mpc'.format(b))
    print('  difference:\t{:.1f}%'.format((abs(b-a)) * 100 / b))

masses = [1E10,1E13,1E15]
redshifts = [0,0.5,1,2]

for z in redshifts:
    for m in masses:
        compareRadii(m, z)

In [ ]:
####################################################
# Get a feel for halo masses for the simulation
####################################################
p = BolshoiProvider()
halos = p.get_halos(0)
print('Max: {:.2e}'.format(max(halos['Mvir'])))
print(np.percentile(halos['Mvir'], [50,80,95,99,99.9,99.99,99.999]))


In [ ]:
###################################################################################################
#Simplified code that calculates the mass within the analytic profile 
#Currently it does this for fire and percipitation but can be easily generalized
#
#The vertical lines are the virial radius
###################################################################################################

redshift = 0
#df = provider.get_halos(redshift)
#df, bin_markers = create_halo_array_for_convolution(df, 10**10, 9*10**15, 30)

massfactor =   1/cosmo.fb*mu*mprot/msun*(Mpc)**3

fp = FireProfile()
percip = PrecipitationProfile()
color = ['black','red', 'orange', 'blue', 'green', 'cyan', 'magenta', 'yellow', 'brown']
nummass=6
i=0
for Mvir_avg in np.logspace(10, 15, nummass):

    rvir = halo.comoving_rvir(cosmo, Mvir_avg, redshift)
    #rvalsFire, densityFire = fp.get_analytic_profile(Mvir_avg, redshift)
    rvalsPercip, densityPercip = percip.get_analytic_profile(Mvir_avg, redshift)


    #totalmassanalyticFire = 4.*np.pi*np.sum(rvalsFire[1:]**2*densityFire[1:] *(rvalsFire[1:] - rvalsFire[:-1]))
    totalmassanalyticPercip = 4.*np.pi*np.sum(rvalsPercip[1:]**2*densityPercip[1:]*(rvalsPercip[1:] - rvalsPercip[:-1]))
    
    #totalmassanalyticFire *= massfactor
    totalmassanalyticPercip *= massfactor
    #print(np.log10(Mvir_avg), "massFire in 1e12 = ", totalmassanalyticFire/1e12, " truth = ", Mvir_avg/1e12, " ratio =", totalmassanalyticFire/Mvir_avg)
    print(np.log10(Mvir_avg), "massPercip in 1e12 = ", totalmassanalyticPercip/1e12,  " truth = ",  Mvir_avg/1e12,  " ratio =", totalmassanalyticPercip/Mvir_avg)
    

    #plt.loglog(rvalsFire, densityFire, color=color[i%nummass])
    plt.loglog(rvalsPercip, densityPercip, '--', color=color[i%nummass])
    plt.axvline(rvir, color=color[i%nummass])
    i+=1
        

plt.xlim([.01,3])

In [ ]:
# Analyze where mass is going in Precipitation Profile

redshift = 0
massfactor =  1/cosmo.fb*mu*mprot/msun*(Mpc)**3
p = PrecipitationProfile()

Mvir = np.logspace(np.log10(DEFAULT_MIN_MASS),np.log10(DEFAULT_MAX_MASS), DEFAULT_MASS_BIN_COUNT)

for m in Mvir:

    rvir_kpc = 1000*halo.comoving_rvir(cosmo, m, redshift) # comoving radius

    print("Mass  {:.1e}".format(m))
    n1, n2, xi1, xi2, neconstant, XRvir, rmax = p.get_precipitation_params(np.log10(m), rvir_kpc, redshift, True)
    #print("3Rvir {:.1f}kpc".format(3*rvir_kpc))
    #print("rmax  {:.1f} kpc".format(rmax))
    #print(" n1 = {:.1e} cm^-3; n2 = {:.1e} cm^-3".format(n1,n2))


In [ ]:
# Mass Bin Analysis
a = np.logspace(np.log10(DEFAULT_MIN_MASS),np.log10(DEFAULT_MAX_MASS), DEFAULT_MASS_BIN_COUNT)
print(a)
#delta = 1.21164168


In [ ]:
# Radial values (in kpc) for a DM vs R profile
provider = BolshoiProvider()
res = 32
date = '2022-04-04'
grid_size = 1024 * res
DM_vs_R = loadArray('STH{}_256_{}_DMvsR_prof'.format(res, date))            
cellsize = (provider.Lbox/grid_size) 
x_axis = radial_distances_kpc(cellsize, DM_vs_R.shape[1])
saveArray('radial_distance_kpc_res32k', x_axis)

loadArray('radial_distance_kpc_res32k')

In [ ]:
loadArray('radial_distance_kpc_res32k')

In [ ]:

# # Run a config, generate DM vs R, visualize radial profiles for analysis
provider = BolshoiProvider()
series = []
resolution = 8
grid_size = provider.halofieldresolution*resolution
load = True
config = Configuration(PrecipitationProfile(), 1, provider=provider, resolution=resolution, den_grid_size=256)
config.run(load_from_files=load)
config.generate_DM_vs_radius_profile(load_from_files=load)
config.generate_profile_of_masks(load_from_files=load)
series.append((config.DM_vs_R1, config.mask_profiles, 'Precipitation', 'c','-'))

vir_rad_ar = config.get_virial_radii()
avg_mass_ar = config.get_halo_masses()

M_chosen = range(0, 40, 3) # plot every 3rd mass bin to see evolution
ymax = (np.arange(0, 40, 3) ** 1.5) + 40
xmin = 25
xmax = 1999
make_DM_vs_Rad_profiles_plots(series, False, True, xmin, xmax, resolution, grid_size, M_chosen, vir_rad_ar, provider, avg_mass_ar, ymax, 'precip_analysis')
